# Import

In [8]:
import pandas as pd
import numpy as np
import holidays
import requests
import os
import re
no_holidays = holidays.NO()

# Parametre & stier

In [9]:
DATA_PATH = "../data/raw_data/historical_flights.csv"
PREDICTION_PATH="../data/raw_data/scheduled_october2025.csv"


# Hjelpefunksjoner

In [10]:
def load_flights(path: str, prediction: bool) -> pd.DataFrame:
    df = pd.read_csv(path)
    if prediction:
        for col in ["std", "sta"]:
            df[col] = pd.to_datetime(df[col], errors="coerce")

    else:
        for col in ["std", "sta", "atd", "ata"]:
            df[col] = pd.to_datetime(df[col], errors="coerce")
    return df

def handle_wrong_times(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    n0 = len(df)

    # 1) Planlagt varighet
    df['duration'] = df['sta'] - df['std']
    df1 = df[(df['duration'] >= pd.Timedelta(0)) &
             (df['duration'] <= pd.Timedelta(hours=10))].copy()
    n1 = len(df1)

    # 2) Ekstremt tidlige avvik (delay < -500 min)
    dep_too_early = df1["atd"].notna() & ((df1["atd"] - df1["std"]) < pd.Timedelta(minutes=-500))
    arr_too_early = df1["ata"].notna() & ((df1["ata"] - df1["sta"]) < pd.Timedelta(minutes=-500))
    df2 = df1.loc[~(dep_too_early | arr_too_early)].copy()
    n2 = len(df2)

    print(f"Totalt: {n0}")
    print(f"  Fjernet på varighet: {n0 - n1}")
    print(f"  Fjernet på ekstremt tidlig dep/arr: {n1 - n2}")
    print(f"Beholdt: {n2}")

    return df2

def build_full_grid(df: pd.DataFrame, prediction:bool) -> pd.DataFrame:
    groups = pd.concat([df["dep_airport_group"], df["arr_airport_group"]]).dropna().unique()
    if prediction:
        tmin = pd.to_datetime(df[["std","sta"]].min(numeric_only=False).min()).floor("h")
        tmax = pd.to_datetime(df[["std","sta"]].max(numeric_only=False).max()).ceil("h")
    else:
        tmin = pd.to_datetime(df[["std","sta","atd","ata"]].min(numeric_only=False).min()).floor("h")
        tmax = pd.to_datetime(df[["std","sta","atd","ata"]].max(numeric_only=False).max()).ceil("h")
    all_hours = pd.date_range(tmin, tmax, freq="h")
    return pd.MultiIndex.from_product([groups, all_hours], names=["airport_group","timestamp"]).to_frame(index=False)


def make_intervals(df: pd.DataFrame, actual: bool = True) -> pd.DataFrame:

    if actual:
        dep = df.dropna(subset=["atd"]).copy()
        dep["start"] = dep["atd"] - pd.to_timedelta(15, "m")
        dep["end"]   = dep["atd"] + pd.to_timedelta(8, "m")


        arr = df.dropna(subset=["ata"]).copy()
        arr["start"] = arr["ata"] - pd.to_timedelta(16, "m")
        arr["end"]   = arr["ata"] + pd.to_timedelta(5, "m")


    else:
        dep = df.dropna(subset=["std"]).copy()
        dep["start"] = dep["std"] - pd.to_timedelta(15, "m")
        dep["end"]   = dep["std"] + pd.to_timedelta(8, "m")

        arr = df.dropna(subset=["sta"]).copy()
        arr["start"] = arr["sta"] - pd.to_timedelta(16, "m")
        arr["end"]   = arr["sta"] + pd.to_timedelta(5, "m")


    dep["airport_group"] = dep["dep_airport_group"]
    dep["type"] = "departure"
    arr["airport_group"] = arr["arr_airport_group"]
    arr["type"] = "arrival"

    intervals = pd.concat([dep, arr], ignore_index=True)
    intervals = intervals.dropna(subset=["airport_group"])
    return intervals


def expand_to_hours(intervals: pd.DataFrame) -> pd.DataFrame:
    rows = []
    for _, row in intervals.iterrows():
        hour_start = row["start"].floor("h")
        hour_end = row["end"].floor("h")
        hours = pd.date_range(hour_start, hour_end, freq="h")
        for h in hours:
            rows.append({**row, "timestamp": h})
    return pd.DataFrame(rows)

def hourly_overlap_group(group: pd.DataFrame) -> pd.DataFrame:
    hour = group["timestamp"].iloc[0]
    airport = group["airport_group"].iloc[0]
    events = []
    for _, row in group.iterrows():
        events.append((row["start"], +1))
        events.append((row["end"], -1))
    events.sort()
    active, overlap = 0, 0
    for _, change in events:
        active += change
        if active > 1:
            overlap = 1
            break
    return pd.DataFrame([{"airport_group": airport, "timestamp": hour, "target": overlap}])




def make_hourly_features_from(intervals_any: pd.DataFrame) -> pd.DataFrame:
    """
    Feature-aggregater, men generisk (kan bruke actual ELLER scheduled intervaller).
    """
    df = intervals_any.copy()
    df["duration_min"] = ((df["sta"] - df["std"]).dt.total_seconds() / 60)
    
    if "flight_id" in df:
        df["airline"] = df["flight_id"].astype(str).str.extract(r"^([A-Za-z]+)")
    df["airline"] = ""

    feats = df.groupby(["airport_group", "timestamp"]).agg(
        flights_cnt     = ("flight_id", "count"),
        avg_duration    = ("duration_min", "mean"),
        max_duration    = ("duration_min", "max"),
        passenger_share = ("service_type", lambda x: (x == "J").mean()),
        cargo_share     = ("service_type", lambda x: (x == "P").mean()),
        charter_share   = ("service_type", lambda x: (x == "C").mean()),
        airline = ("airline", lambda x: x.mode().iloc[0] if not x.mode().empty else "")
    ).reset_index()

    return feats
def add_time_features(feats: pd.DataFrame) -> pd.DataFrame:
    feats["dow"]     = feats["timestamp"].dt.dayofweek
    feats["holiday"] = feats["timestamp"].apply(lambda x: x.date() in no_holidays)
    feats["month"]   = feats["timestamp"].dt.month
    feats["hournum"] = feats["timestamp"].dt.hour
    feats["weekend"] = (feats["dow"] >= 5).astype(int)

    feats["date"] = feats["timestamp"].dt.normalize()

    feats["daily_flights_cnt"] = feats.groupby(
        ["airport_group", "date"]
    )["flights_cnt"].transform("sum")

    feats = feats.sort_values(["airport_group", "timestamp"])
    
    return feats

def add_prev_next(feats: pd.DataFrame) -> pd.DataFrame:
    feats = feats.sort_values(["airport_group", "timestamp"])
    feats["flights_cnt_prev"] = feats.groupby("airport_group")["flights_cnt"].shift(1)
    feats["flights_cnt_next"] = feats.groupby("airport_group")["flights_cnt"].shift(-1)
    feats[["flights_cnt_prev", "flights_cnt_next"]] = feats[["flights_cnt_prev", "flights_cnt_next"]].fillna(0).astype(int)
    return feats


# Last rådata

In [11]:
df_raw = load_flights(DATA_PATH, prediction=False)
df_pred = load_flights(PREDICTION_PATH, prediction=True)
df_raw

,flight_id,dep_airport,dep_airport_group,arr_airport,arr_airport_group,service_type,std,sta,cancelled,atd,ata
0,WF149,HOV,B,OSL,NaN,J,2018-01-02 16:40:00,2018-01-02 17:15:00,0,NaT,2018-01-02 18:53:00
1,WF722,OSL,NaN,MJF,D,J,2018-01-28 13:04:00,2018-01-28 14:50:00,0,NaT,NaT
2,WF188,FDE,A,OSL,NaN,J,2018-04-07 07:10:00,2018-04-07 08:10:00,0,NaT,2018-04-07 07:55:00
3,WF176,HOV,B,OSL,NaN,J,2018-04-07 11:00:00,2018-04-07 12:05:00,0,NaT,2018-04-07 12:00:00
4,WF148,HOV,B,OSL,NaN,J,2018-04-30 08:25:00,2018-04-30 09:26:00,0,NaT,2018-04-30 09:36:00
...,...,...,...,...,...,...,...,...,...,...,...
410437,WF153,SOG,A,HOV,B,J,2025-05-03 09:25:00,2025-05-03 10:00:00,0,2025-05-03 09:47:00,2025-05-03 10:19:00
410438,WF153,BGO,NaN,SOG,A,J,2025-05-03 08:35:00,2025-05-03 09:10:00,0,2025-05-03 08:29:00,2025-05-03 09:18:00
410439,WF158,OSL,NaN,HOV,B,J,2025-05-03 14:40:00,2025-05-03 15:50:00,0,2025-05-03 14:35:00,2025-05-03 15:39:00
410440,WF721,SSJ,D,TRD,NaN,J,2025-05-03 08:50:00,2025-05-03 09:45:00,0,2025-05-03 08:49:00,2025-05-03 09:39:00


# Rens og filter

In [12]:
df = handle_wrong_times(df_raw)

Totalt: 410442
  Fjernet på varighet: 86
  Fjernet på ekstremt tidlig dep/arr: 97
Beholdt: 410259


# Bygger full grid for alle timer i intervallet

In [13]:
grid = build_full_grid(df, prediction=False)
grid_pred=build_full_grid(df_pred, prediction=True)

# Intervaller (actual) + hourly overlap (actual)

In [14]:
intervals_actual = make_intervals(df, actual=True)
intervals_actual_expanded = expand_to_hours(intervals_actual)


hourly_actual = (
    intervals_actual_expanded
    .groupby(["airport_group", "timestamp"], group_keys=False)
    .apply(hourly_overlap_group)
    .rename(columns={"target": "target_actual"})
)

hourly_actual.head()

/tmp/ipykernel_9590/1819245093.py:8: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(hourly_overlap_group)


,airport_group,timestamp,target_actual
0,A,2018-01-01 12:00:00,0
0,A,2018-01-01 13:00:00,1
0,A,2018-01-01 14:00:00,0
0,A,2018-01-01 16:00:00,1
0,A,2018-01-01 17:00:00,1


# Intervaller (scheduled) + hourly overlap (scheduled)

In [15]:
intervals_sched = make_intervals(df, actual=False)
intervals_sched_expanded = expand_to_hours(intervals_sched)

hourly_sched = (
    intervals_sched_expanded
    .groupby(["airport_group", "timestamp"], group_keys=False)
    .apply(hourly_overlap_group)
    .rename(columns={"target": "target_sched"})
)




/tmp/ipykernel_9590/3526365751.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(hourly_overlap_group)


# Prediction Data Intervaller (scheduled) + hourly overlap (scheduled) 

In [16]:
intervals_pred = make_intervals(df_pred, actual=False)
intervals_pred_expanded = expand_to_hours(intervals_pred)

hourly_pred=(
    intervals_pred_expanded
    .groupby(["airport_group", "timestamp"], group_keys=False)
    .apply(hourly_overlap_group)
    .rename(columns={"target": "target_sched"})
)

/tmp/ipykernel_9590/2247729525.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(hourly_overlap_group)


# Merge targets dataset

In [17]:
hourly = (grid
          .merge(hourly_actual, on=["airport_group","timestamp"], how="left")
          .merge(hourly_sched,  on=["airport_group","timestamp"], how="left"))
hourly[["target_actual","target_sched"]] = hourly[["target_actual","target_sched"]].fillna(0).astype(int)
hourly.head()


,airport_group,timestamp,target_actual,target_sched
0,B,2018-01-01 07:00:00,0,0
1,B,2018-01-01 08:00:00,0,0
2,B,2018-01-01 09:00:00,0,0
3,B,2018-01-01 10:00:00,0,1
4,B,2018-01-01 11:00:00,1,1


# Merge targets prediction dataset

In [18]:
hourly_pred=grid_pred.merge(hourly_pred, on=["airport_group","timestamp"], how="left")
hourly_pred[["target_sched"]] = hourly_pred[["target_sched"]].fillna(0).astype(int)
hourly_pred.head()

,airport_group,timestamp,target_sched
0,G,2025-10-01 03:00:00,1
1,G,2025-10-01 04:00:00,0
2,G,2025-10-01 05:00:00,0
3,G,2025-10-01 06:00:00,1
4,G,2025-10-01 07:00:00,0


# Feature engineering

# Feature-agg 

In [19]:
feat_raw = make_hourly_features_from(intervals_sched_expanded.copy())
feat_raw.head()

,airport_group,timestamp,flights_cnt,avg_duration,max_duration,passenger_share,cargo_share,charter_share,airline
0,A,2018-01-01 11:00:00,3,56.666667,60.0,1.0,0.0,0.0,
1,A,2018-01-01 12:00:00,3,46.666667,60.0,1.0,0.0,0.0,
2,A,2018-01-01 15:00:00,2,60.000000,60.0,1.0,0.0,0.0,
3,A,2018-01-01 16:00:00,4,46.250000,65.0,1.0,0.0,0.0,
4,A,2018-01-01 17:00:00,4,42.500000,60.0,1.0,0.0,0.0,


# Feature-agg pred

In [20]:
feat_pred_raw = make_hourly_features_from(intervals_pred_expanded.copy())
feat_pred_raw.head()

,airport_group,timestamp,flights_cnt,avg_duration,max_duration,passenger_share,cargo_share,charter_share,airline
0,A,2025-10-01 03:00:00,2,47.500000,65.0,1.0,0.0,0.0,
1,A,2025-10-01 04:00:00,6,40.833333,65.0,1.0,0.0,0.0,
2,A,2025-10-01 06:00:00,3,51.666667,65.0,1.0,0.0,0.0,
3,A,2025-10-01 07:00:00,6,41.666667,65.0,1.0,0.0,0.0,
4,A,2025-10-01 08:00:00,2,47.500000,65.0,1.0,0.0,0.0,


## Merger med full grid for alle timer og fyller inn verdier for timer der det ikke er noen fly

In [70]:
feat_full = grid.merge(feat_raw, on=["airport_group","timestamp"], how="left")
feat_full_pred = grid_pred.merge(feat_pred_raw, on=["airport_group","timestamp"], how="left")

for df_ in (feat_full, feat_full_pred):
    for c in ["flights_cnt","avg_duration","max_duration",
              "passenger_share","cargo_share","charter_share","airline", "hour"]:
        if c in df_.columns:
            if c == "flights_cnt":
                df_[c] = df_[c].fillna(0).astype(int)
            elif c == "airline":
                df_[c] = df_[c].fillna("")
            else:
                df_[c] = df_[c].fillna(0)

## Time features

In [71]:
feat_full      = add_time_features(feat_full)
feat_full_pred = add_time_features(feat_full_pred)

## Next and previous hour flights count

In [72]:
feat_full     = add_prev_next(feat_full)
feat_full_pred = add_prev_next(feat_full_pred)

## Vær

#### Add locations for airports

In [73]:
airport_locations_df = pd.read_csv("../data/raw_data/airports.csv")
airport_groups_df = pd.read_csv("../data/raw_data/airportgroups.csv")

airport_locations_df = airport_locations_df.merge(airport_groups_df, left_on="iata_code", right_on="airport")
airport_locations_df = airport_locations_df.groupby(by = "airport_group")[["latitude_deg", "longitude_deg"]].mean()

#### Find weather

In [74]:
## TODO: USE AIRPORT LOCATION ABOVE
sensors = {}

for (group, location) in airport_locations_df.iterrows():
    endpoint = "https://frost.met.no/sources/v0.jsonld"
    parameters = {
        "types": "SensorSystem",
        "geometry": f"nearest(POINT({location["longitude_deg"]} {location["latitude_deg"]}))"
    }
    r = requests.get(endpoint, parameters, auth=(os.getenv("FROST_ID"),''))
    resp = r.json()

    sensors[group] = resp

for key in sensors.keys():
    sensors[key] = sorted(sensors[key]["data"], key=lambda v: v["distance"])

KeyError: 'data'

In [106]:
weather = {}
values = [None] * 465143

count = 0

for (i, row) in feat_full.iterrows():
    key = row["timestamp"].strftime("%Y-%m-%d")

    if (row["airport_group"] != "A"):
        continue

    if weather.get(row["airport_group"]) == None:
        weather[row["airport_group"]] = {}

    if weather.get(row["airport_group"]).get(key) == None:
        endpoint = 'https://frost.met.no/observations/v0.jsonld'
        parameters = {
            'sources': sensors[row["airport_group"]][0]["id"],
            'elements': 'mean(air_temperature P1D),sum(precipitation_amount P1D),mean(wind_speed P1D)',
            'referencetime': key,
        }
        r = requests.get(endpoint, parameters, auth=(os.getenv("FROST_ID"),''))
    
        if r.status_code == 200:
            json = r.json()
    
            k = json['data'][0]['observations'][0]['elementId']
            v = json['data'][0]['observations'][0]['value']
            
            weather[row["airport_group"]][key] = float(v)
            count += 1

            print(f"{count} / 66449")
        else:
            weather[row["airport_group"]][key] = float('nan')
    
    feat_full.at[i, "weather"] = weather.get(row["airport_group"]).get(key)

feat_full.head(10)

1 / 66449
2 / 66449
3 / 66449
4 / 66449
5 / 66449
6 / 66449
7 / 66449
8 / 66449
9 / 66449
10 / 66449
11 / 66449
12 / 66449
13 / 66449
14 / 66449
15 / 66449
16 / 66449
17 / 66449
18 / 66449
19 / 66449
20 / 66449
21 / 66449
22 / 66449
23 / 66449
24 / 66449
25 / 66449
26 / 66449
27 / 66449
28 / 66449
29 / 66449
30 / 66449
31 / 66449
32 / 66449
33 / 66449
34 / 66449
35 / 66449
36 / 66449
37 / 66449
38 / 66449
39 / 66449
40 / 66449
41 / 66449
42 / 66449
43 / 66449
44 / 66449
45 / 66449
46 / 66449
47 / 66449
48 / 66449
49 / 66449
50 / 66449
51 / 66449
52 / 66449
53 / 66449
54 / 66449
55 / 66449
56 / 66449
57 / 66449
58 / 66449
59 / 66449
60 / 66449
61 / 66449
62 / 66449
63 / 66449
64 / 66449
65 / 66449
66 / 66449
67 / 66449
68 / 66449
69 / 66449
70 / 66449
71 / 66449
72 / 66449
73 / 66449
74 / 66449
75 / 66449
76 / 66449
77 / 66449
78 / 66449
79 / 66449
80 / 66449
81 / 66449
82 / 66449
83 / 66449
84 / 66449
85 / 66449
86 / 66449
87 / 66449
88 / 66449
89 / 66449
90 / 66449
91 / 66449
92 / 664

KeyboardInterrupt: 

# Samle datasett + split

In [19]:
dataset = hourly.merge(feat_full, on=["airport_group", "timestamp"], how="left").sort_values("timestamp")

pred= hourly_pred.merge(feat_full_pred,on=["airport_group","timestamp"], how="left").sort_values("timestamp")

dataset["timestamp"] = pd.to_datetime(dataset["timestamp"])
pred["timestamp"]    = pd.to_datetime(pred["timestamp"])

CUTOFF_VAL  = pd.Timestamp("2025-01-01")  
CUTOFF_TEST = pd.Timestamp("2025-04-01")

train = dataset[dataset["timestamp"] <  CUTOFF_VAL].copy()
val   = dataset[(dataset["timestamp"] >= CUTOFF_VAL) & 
                (dataset["timestamp"] <  CUTOFF_TEST)].copy()
test  = dataset[dataset["timestamp"] >= CUTOFF_TEST].copy()



print(train.shape, val.shape, test.shape, pred.shape)


(429527, 20) (15120, 20) (20496, 20) (5026, 19)


# lagre mellomfiler

In [20]:
train.to_csv('../data/processed_data/train.csv', index=False)
val.to_csv('../data/processed_data/val.csv', index=False)
test.to_csv('../data/processed_data/test.csv', index=False)
pred.to_csv('../data/processed_data/predict_oct2025.csv', index=False)